In [1]:
import os
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=24'

#from jax.config import config; config.update("jax_enable_x64", True)

%load_ext autoreload
%autoreload 2

from brax import envs
from brax.io import html
from brax.training import normalization

import flax
import jax
import jax.numpy as jnp
from brax.envs import create_fn

from IPython.display import HTML, clear_output

import optax

import matplotlib.pyplot as plt
import numpy as np

from controllers import GruController, MlpController, LinearController
from common import do_local_apg, add_guassian_noise, add_guassian_noise_mixed_std, do_one_rollout, cem_apg

from functools import partial


jax.config.update('jax_platform_name', 'cpu')

def visualize(sys, qps, height=480):
  """Renders a 3D visualization of the environment."""
  return HTML(html.render(sys, qps, height=height))

len(jax.devices())

1

In [ ]:
env_name = "inverted_pendulum_swingup"  # @param ['ant', 'humanoid', 'fetch', 'grasp', 'halfcheetah', 'walker2d, 'ur5e', 'reacher', bball_1dof]

episode_length = 500
action_repeat = 1
env_fn = create_fn(env_name = env_name, action_repeat=action_repeat, batch_size=None, auto_reset=False)
env = env_fn()

policy_size = int(2**jnp.ceil(jnp.log2(env.observation_size*4)))
print(policy_size)
policy = GruController(env.observation_size, env.action_size, 16)

inference_fn, params, metrics = cem_apg(env_fn,
                                        100,
                                        episode_length = episode_length,
                                        action_repeat = action_repeat,
                                        apg_epochs = 75,
                                        cem_epochs = 1,
                                        batch_size = 1,
                                        truncation_length = None,
                                        learning_rate = 5e-4,
                                        clipping = 1e9,
                                        initial_std = 0.01,
                                        num_elite = 8,
                                        eps = 0.0,
                                        normalize_observations=True,
                                        policy = policy
                                       )






64
